<img src = 'logo.png'>

In [1]:
import pandas as pd
import numpy as np

import datetime as dt

import warnings
warnings.filterwarnings(action='ignore')

import os
import shutil # shutil.rmtree

#-------------------- visualization (edcoding, grid) -----------

import matplotlib.pyplot as plt
import seaborn as sns

#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
# sns.set()


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel 
from ast import literal_eval # iterable(string) -> object


# ------------- tensorflow & keras -----------------
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation                 #-------------FC
from keras.layers import Conv2D, MaxPooling2D,Flatten      #-------------CNN
from keras.layers import LSTM                              #-------------RNN
from keras.preprocessing.image import ImageDataGenerator   #-------------Augmentation
from keras.preprocessing.image import array_to_img, img_to_array, load_img # ㄴ flow



from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint  #------------ callback

# --------------------- etc. -----------------------
from PIL import Image


# ----------------- fixing seed --------------------
np.random.seed(1024)
tf.random.set_seed(1024)

In [3]:
#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

# dataload

In [2]:
mdf = pd.read_csv("./dataset/movies_metadata_2.csv")

In [5]:
print(mdf.shape)
print(mdf.info())
mdf.head(2)

(45466, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  obj

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995


## id 중 오데이터 drop(날짜)

In [6]:
idx= mdf[mdf['id'].str.len()>6].index
mdf = mdf.drop(idx, axis=0)

In [7]:
mdf['id']=mdf['id'].astype('int')

# Weighted Ranking(Top %) : df['wr']
rul : https://www.quora.com/How-does-IMDbs-rating-system-work <br>
<img src = 'rvmc.png' width=600>

* R : 개별 영화 평점
* v : 개별 영화의 투표수
* m : 상위 % 안에 들어야 하는 최소 투표수
* C : 전체 영화의 평균 평점


In [8]:
C = mdf['vote_average'].mean()
C

5.618207215134184

In [9]:
m = mdf['vote_count'].quantile(0.95)
m

434.0

In [10]:
C = mdf['vote_average'].mean()
m = mdf['vote_count'].quantile(0.95)


def my_calc_wr(mdf):
    R = mdf['vote_average']
    v = mdf['vote_count']
    WR = (v / (v+m) * R + (m/(v+m)*C))
    return WR

In [11]:
mdf['wr'] = mdf.apply(my_calc_wr, axis=1)

In [12]:
mdf['wr']

0        7.545529
1        6.704602
2        5.772437
3        5.653209
4        5.641519
           ...   
45461    5.614487
45462    5.641423
45463    5.593413
45464    5.618207
45465    5.618207
Name: wr, Length: 45463, dtype: float64

In [13]:
df5 = mdf[mdf['vote_count'] > mdf['vote_count'].quantile(0.95)][['id', 'title','genres', 'vote_average', 'vote_count', 'year', 'wr']]

In [14]:
df5.shape

(2268, 7)

In [15]:
df5.head(2)

,id,title,genres,vote_average,vote_count,year,wr
0,862,Toy Story,"['Animation', 'Comedy', 'Family']",7.7,5415.0,1995,7.545529
1,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']",6.9,2413.0,1995,6.704602


In [16]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2268 entries, 0 to 45014
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            2268 non-null   int32  
 1   title         2268 non-null   object 
 2   genres        2268 non-null   object 
 3   vote_average  2268 non-null   float64
 4   vote_count    2268 non-null   float64
 5   year          2268 non-null   object 
 6   wr            2268 non-null   float64
dtypes: float64(3), int32(1), object(3)
memory usage: 132.9+ KB


In [17]:
df5.loc[0, 'genres'], type(df5.loc[0, 'genres']) # object 꼴이지만 str

("['Animation', 'Comedy', 'Family']", str)

## search

In [18]:
# df5['Comedy' in df5['genres']] # ---- ERROR!
df5[df5['genres'].str.contains('Comedy')].head()

,id,title,genres,vote_average,vote_count,year,wr
0,862,Toy Story,"['Animation', 'Comedy', 'Family']",7.7,5415.0,1995,7.545529
17,5,Four Rooms,"['Crime', 'Comedy']",6.5,539.0,1995,6.106682
18,9273,Ace Ventura: When Nature Calls,"['Crime', 'Comedy', 'Adventure']",6.1,1128.0,1995,5.966134
33,9598,Babe,"['Fantasy', 'Drama', 'Comedy', 'Family']",6.0,756.0,1995,5.860758
38,9603,Clueless,"['Comedy', 'Drama', 'Romance']",6.9,828.0,1995,6.459193


In [19]:
def my_calc_wr(mdf):
    R = mdf['vote_average']
    v = mdf['vote_count']
    WR = (v / (v+m) * R + (m/(v+m)*C))
    return WR

mdf= pd.read_csv('./dataset/movies_metadata_2.csv')
idx = mdf[mdf['id'].str.len()>6].index
mdf = mdf.drop(idx, axis = 0)
mdf['id'] = mdf['id'].astype('int')


In [20]:
def my_search_wr_by_genres(search_genres = 'Family', percent = 0.95):
    C = mdf['vote_average'].mean()
    m = mdf['vote_count'].quantile(percent)
    mdf['wr'] = mdf.apply(my_calc_wr, axis = 1)
    
    df5 = mdf[mdf['vote_count'] > m][['id', 'title','genres', 'vote_average', 'vote_count', 'year', 'wr']]
    df5 = df5.sort_values('wr', ascending = False)
    
    return df5[df5['genres'].str.contains(search_genres)].head()
# 검색한 것의 weight ranking 중에서 장르 검색하면 그에 해당하는 데이터프레임.head() 나오게
    

In [21]:
resdf = my_search_wr_by_genres('Fantasy', 0.97)
resdf

,id,title,genres,vote_average,vote_count,year,wr
5481,129,Spirited Away,"['Fantasy', 'Adventure', 'Animation', 'Family']",8.3,3968.0,2001,8.035598
7000,122,The Lord of the Rings: The Return of the King,"['Adventure', 'Fantasy', 'Action']",8.1,8226.0,2003,7.975624
3030,497,The Green Mile,"['Fantasy', 'Drama', 'Crime']",8.2,4166.0,1999,7.956413
4863,120,The Lord of the Rings: The Fellowship of the Ring,"['Adventure', 'Fantasy', 'Action']",8.0,8892.0,2001,7.889160
5814,121,The Lord of the Rings: The Two Towers,"['Adventure', 'Fantasy', 'Action']",8.0,7641.0,2002,7.871988


# Review based

## 결측처리

In [22]:
mdf['tagline'] = mdf['tagline'].fillna('')

In [23]:
mdf['overview'] = mdf['overview'].fillna('')

In [24]:
mdf[['overview', 'tagline']].tail(2)

,overview,tagline
45464,"In a small town live two brothers, one a minister and the other one a hunchback painter of the chapel who lives with his wife. One dreadful and stormy night, a stranger knocks at the door asking for shelter. The stranger talks about all the good things of the earthly life the minister is missing because of his puritanical faith. The minister comes to accept the stranger's viewpoint but it is others who will pay the consequences because the minister will discover the human pleasures thanks to, ehem, his sister- in -law… The tormented minister and his cuckolded brother will die in a strange accident in the chapel and later an infant will be born from the minister's adulterous relationship.",
45465,"50 years after decriminalisation of homosexuality in the UK, director Daisy Asquith mines the jewels of the BFI archive to take us into the relationships, desires, fears and expressions of gay men and women in the 20th century.",


In [25]:
mdf['view_tag'] = mdf['overview'] + mdf['tagline']

In [26]:
idx = mdf[mdf['view_tag']==''].index
mdf = mdf.drop(idx, axis = 0)

In [27]:
mdf.shape # 데이터 크기가 너무 커서 연결 하고 간다.

(44517, 27)

## Data Load

In [28]:
ldf = pd.read_csv('./dataset/links_small.csv')

In [29]:
print(ldf.shape)
print(ldf.info())
ldf.head(2)

(9125, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB
None


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


### 결측처리

In [30]:
ldf.isna().sum()

movieId     0
imdbId      0
tmdbId     13
dtype: int64

In [31]:
ldf = ldf.dropna()
print(ldf.shape)

(9112, 3)


In [32]:
ldf['tmdbId'] = ldf['tmdbId'].astype('int')

In [33]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9112 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   movieId  9112 non-null   int64
 1   imdbId   9112 non-null   int64
 2   tmdbId   9112 non-null   int32
dtypes: int32(1), int64(2)
memory usage: 249.2 KB


### mdf + ldf join

In [34]:
mldf = pd.merge(mdf, ldf, left_on='id', right_on='tmdbId') # how = 'inner'
mldf.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,wr,view_tag,movieId,imdbId,tmdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,1995,7.545529,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",1,114709,862
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,6.704602,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.Roll the dice and unleash the excitement!",2,113497,8844
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0,1995,5.772437,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll sca

In [35]:
mldf[['view_tag', 'id']]

,view_tag,id
0,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",862
1,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.Roll the dice and unleash the excitement!",8844
2,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.Still Yelling. Still Fighting. Still Ready for Love.",15602
3,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.Friends are the people who let you be yourself... and never let you forget it.",31357
4,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.Just When His World Is Back To Normal... He's In For The Surprise Of His Life!",11862
...,...,...
9082,A man must cope with the loss of his wife and the obsolescence of his job before finding redemption by becoming a role model to an equally lost 13-year-old.,159550
9083,"Rustom Pavri, an honourable officer of the Indian Navy shoots his friend Vikram to death after discovering that his wife Cynthia had an affair with the rich businessman. The Commander surrenders himself to the Police immediately and admits to have killed Vikram but pleads 'not guilty' in court. Is he convicted or acquitted?Decorated Officer. Devoted Family Man. Defending His Honor. His Name Is RUSTOM.",392572
9084,"Village lad Sarman is drawn to big, bad Mohenjo Daro - and its mascot Chaani. But Chaani must wed Munja, son of Mohenjo Daro's ruler, Maham. Will Sarman find love - and more - in Mohenjo Daro?",402672
9085,"From the mind behind Evangelion comes a hit larger than life. When a massive, gilled monster emerges from the deep and tears through the city, the government scrambles to save its citizens. A rag-tag team of volunteers cuts through a web of red tape to uncover the monster's weakness and its mysterious ties to a foreign superpower. But time is not on their side - the greatest catastrophe to ever befall the world is about to evolve right before their very eyes.A god incarnate. A city doomed.",315011


In [36]:
mldf.shape

(9087, 30)

## 코사인 유사도

In [37]:
from numpy.linalg import norm ## ||A|| : norm

In [38]:
def my_cos_sim(dataframe, id1, id2):
    A = dataframe[dataframe['id']==id1]['view_tag']
    B = dataframe[dataframe['id']==id2]['view_tag']
    
    corpus = [A.values[0], B.values[0]]

    # CountVectorizer든 TfidfVectorizer든 자기 마음대로..
#     cnt_vt = CountVectorizer()
#     res = cnt_vt.fit_transform(corpus) 
    
    tfidf = TfidfVectorizer()
    res =  tfidf.fit_transform(corpus)

    A = res.toarray()[0]
    B = res.toarray()[1]
    
    cos_sim = np.dot(A,B) / (norm(A) * norm(B))
    
    return cos_sim

In [39]:
my_cos_sim(mldf, 9082, 392572)

0.17027932890369468

## Tf-idf

<pre>
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
tfidf = TfidfVectorizer(stop_words='english') #, ngram_range=(1,2), max_df=0.8, min_df=0.2)
tfidf_matrix =  tfidf.fit_transform(mldf['view_tag']) 
# print(tfidf.vocabulary_)
print(tfidf_matrix.shape)

(9087, 30708)


## 유사도 계산
<pre>
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel 

In [41]:
# tfidf_matrix
cos_sim = cosine_similarity(tfidf_matrix)
cos_sim_view_tag = cos_sim
# cos_sim_view_tag = cosine_similarity(tfidf_matrix) # Y 안 주면 X, Y에 대해

In [42]:
cos_sim[0][:20]

array([1.        , 0.01667211, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01673995, 0.02936277, 0.        , 0.        ])

In [43]:
# linear_kernel(tfidf_matrix) # 같은 결과

## 영화 제목 인덱스

In [44]:
keyword = 'Jumanji'
mldf[mldf['title']==keyword].index[0]

1

In [45]:
s = mldf['title']
title_s = pd.Series(s.index, index = s.values) # 값 <--> 인덱스 변경
title_s['Jumanji']

1

In [46]:
def my_search_cossim_by_review(title = "Toy Story", topn=10):
    # 인덱스 출력하기 다른 ver
    s = mldf['title']
    title_s = pd.Series(s.index, index = s.values) # 값 <--> 인덱스 변경
    idx = title_s[title]
    
    # 인덱스 출력하기 다른 ver
    # idx = mldf[mldf['title']==keyword].index[0] # 나중 되면 위 방법이 더 꺼내기 편할듯.
    
    idx_list = pd.Series(cos_sim[idx].reshape(-1)).sort_values(ascending = False).index[1:topn+1] # 0번재는 본인. 1~10번째
    
    title_list = mldf.loc[idx_list,'title'].values
    
    return title_list
    

In [47]:
my_search_cossim_by_review(title = "Heat", topn=5)

array(['Mulholland Falls', 'Kiss Kiss Bang Bang', 'Night Moves',
       'Welcome to L.A.', 'Escape from L.A.'], dtype=object)

In [48]:
# np.__version__

In [49]:
# drop하는 라인마다 reset_index(drop=True)해줬다. // 나는 순서가 달라서 안 해도 에러 안 나는듯

# Actor, Director, ... based

In [50]:
cdf = pd.read_csv("./dataset/credits.csv")
kdf = pd.read_csv("./dataset/keywords.csv")

In [51]:
print(cdf.shape)
print(cdf.info())
cdf.head(2)

(45476, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


cast  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'}, {'cast_id': 19, 'character': 'Hamm (voice)', 'credit_id': '52fe4284c3a36847f8024fa9', 'gender': 2, 'id': 7907, 'name': 'John Ratzenberger', 'order': 5, 'profile_path': '/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg'}, {'cast_id': 20, 'character': 'Bo Peep (voice)', 'credit_id': '52fe4284c3a36847f8024fad', 'gender': 1, 'id': 8873, 'name': 'Annie Potts', 'order': 6, 'profile_path': '/eryXT84RL41jHSJcMy4kS3u9y6w.jpg'}, {'cast_id': 26, 'character': 'Andy (voice)', 'credit_id': '52fe4284c3a36847f8024fc1', 'gender': 0, 'id': 1116442, 'name': 'John Morris', 'order': 7, 'profile_path': '/vYG

In [52]:
print(kdf.shape)
print(kdf.info())
kdf.head(2)

(46419, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB
None


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 10941, 'name': 'disappearance'}, {'id': 15101, 'name': ""based on children's book""}, {'id': 33467, 'name': 'new home'}, {'id': 158086, 'name': 'recluse'}, {'id': 158091, 'name': 'giant insect'}]"


## mldf = mdf + ldf + cdf, jdf : join

In [53]:
temp = pd.merge(mldf, cdf, left_on='id', right_on='id', how = 'inner')
df = pd.merge(temp, kdf, left_on='id', right_on='id', how = 'inner') # cast	crew	keywords 추가됨.

In [54]:
df.shape

(9207, 33)

# 가공

In [55]:
# cast : {'name' : 'Tom Hanks'}
# craw : {'job', 'Director', 'name':'John Lasseter'}
# keywords : {'name': 'jealousy'}

In [56]:
df['keywords'].head()

0                                [{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]
1                                                                                                            [{'id': 10090, 'name': 'board game'}, {'id': 10941, 'name': 'disappearance'}, {'id': 15101, 'name': "based on children's book"}, {'id': 33467, 'name': 'new home'}, {'id': 158086, 'name': 'recluse'}, {'id': 158091, 'name': 'giant insect'}]
2                                                                                                                                                                                                [{'id': 1495, 'name': 'fishing'}, {'id': 12392, 'name': 'best friend'}, {'id': 179431, 'name': 'duringcreditsst

In [57]:
type(df.loc[0, 'keywords']), df.loc[0, 'keywords']

(str,
 "[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]")

In [58]:
df['cast'] = df['cast'].apply(literal_eval)            # 배우
df['crew'] = df['crew'].apply(literal_eval)            # 감독
df['keywords'] = df['keywords'].apply(literal_eval)    # 대표키워드, 소재
df['genres'] = df['genres'].apply(literal_eval)        # 장르

In [59]:
type(df.loc[0, 'keywords'])

list

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9207 entries, 0 to 9206
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9207 non-null   object 
 1   belongs_to_collection  1688 non-null   object 
 2   budget                 9207 non-null   object 
 3   genres                 9207 non-null   object 
 4   homepage               2001 non-null   object 
 5   id                     9207 non-null   int32  
 6   imdb_id                9207 non-null   object 
 7   original_language      9207 non-null   object 
 8   original_title         9207 non-null   object 
 9   overview               9207 non-null   object 
 10  popularity             9207 non-null   object 
 11  poster_path            9204 non-null   object 
 12  production_companies   9207 non-null   object 
 13  production_countries   9207 non-null   object 
 14  release_date           9207 non-null   object 
 15  reve

## 파생변수 : 감독

In [61]:
# 강사님
def get_director(s):
    for dict in s:
        if dict['job'] == 'Director':
            dict['name'] = dict['name'].replace(' ', '')
            return [dict['name'].lower()] # list로 return
        # return np.nan
        
        
df['director'] = df['crew'].apply(get_director)
df['director'].head()

0      [johnlasseter]
1       [joejohnston]
2      [howarddeutch]
3    [forestwhitaker]
4      [charlesshyer]
Name: director, dtype: object

In [62]:
# # csh
# def extract_director(list):
#     for i in range(len(list)):
#         if list[i]['job']=='Director':
#             return list[i]['name']
# mlckdf['direct'].apply(extract_director)

In [63]:
# # 나
# def director(crew):
#     for n in range(len(crew)):
#         if crew[n]['department']=='Directing':
#             director_name = crew[n]['name']
#             return director_name


# df['direct'] = df['crew'].apply(director)

In [64]:
# for i, crew in enumerate(df['crew']):
#     print('Title: ', df.loc[i,'title'], ', ID: ', df.loc[i, 'id'])
#     for n in range(len(crew)):
#         if crew[n]['department']=='Directing':
#             print(crew[n]['department'], crew[n]['name'])
#         # print(crew[n])
    

In [65]:
# # for i in range(len(df['
#                       # ']))

# df['crew'][0][0]

## 파생변수 : 배우

In [66]:
def get_cast(s):
    cast_list= []
    for dict in s:
        dict['name'] = dict['name'].replace(' ', '')
        cast_list.append(dict['name'].lower())
    return cast_list[:3]

In [67]:
df['actor'] = df['cast'].apply(get_cast)

In [68]:
df['actor'].head()

0                  [tomhanks, timallen, donrickles]
1       [robinwilliams, jonathanhyde, kirstendunst]
2          [waltermatthau, jacklemmon, ann-margret]
3    [whitneyhouston, angelabassett, lorettadevine]
4           [stevemartin, dianekeaton, martinshort]
Name: actor, dtype: object

## 파생변수 : 키워드

In [69]:
df['key'] = df['keywords'].apply(get_cast)

In [70]:
df['key'].head()

0                                     [jealousy, toy, boy]
1        [boardgame, disappearance, basedonchildren'sbook]
2              [fishing, bestfriend, duringcreditsstinger]
3    [basedonnovel, interracialrelationship, singlemother]
4                        [baby, midlifecrisis, confidence]
Name: key, dtype: object

## 파생변수 : 장르

In [71]:
df['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

## 파생변수 합친 파생변수 만들기

In [72]:
df['search4'] = df['director'] + df['actor'] + df['key'] + df['genres']
df['search4'].head()

0                                                  [johnlasseter, tomhanks, timallen, donrickles, jealousy, toy, boy, Animation, Comedy, Family]
1          [joejohnston, robinwilliams, jonathanhyde, kirstendunst, boardgame, disappearance, basedonchildren'sbook, Adventure, Fantasy, Family]
2                             [howarddeutch, waltermatthau, jacklemmon, ann-margret, fishing, bestfriend, duringcreditsstinger, Romance, Comedy]
3    [forestwhitaker, whitneyhouston, angelabassett, lorettadevine, basedonnovel, interracialrelationship, singlemother, Comedy, Drama, Romance]
4                                                 [charlesshyer, stevemartin, dianekeaton, martinshort, baby, midlifecrisis, confidence, Comedy]
Name: search4, dtype: object

## 유사도 계산 -------------------review유사도와 상동

In [73]:
df['search4'] = df['search4'].astype('str')

# tfidf = TfidfVectorizer() #, ngram_range=(1,2), max_df=0.8, min_df=0.2)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix =  tfidf.fit_transform(df['search4']) 

# cnt_vt = CountVectorizer(stop_words='english')
# res = cnt_vt.fit_transform(df['search4']) 

# print(tfidf.vocabulary_)
print(tfidf_matrix.shape)

(9207, 18773)


In [74]:
# tfidf_matrix
cos_sim = cosine_similarity(tfidf_matrix) # Y 안 주면 X, Y에 대해
# cos_sim = cosine_similarity(res)

In [75]:
cos_sim

array([[1.        , 0.02231189, 0.00910515, ..., 0.        , 0.        ,
        0.        ],
       [0.02231189, 1.        , 0.        , ..., 0.017819  , 0.01583095,
        0.        ],
       [0.00910515, 0.        , 1.        , ..., 0.01490918, 0.        ,
        0.        ],
       ...,
       [0.        , 0.017819  , 0.01490918, ..., 1.        , 0.02340357,
        0.        ],
       [0.        , 0.01583095, 0.        , ..., 0.02340357, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [76]:
def my_search_cossim_by_search4(title = "Toy Story", topn=10):
    s = df['title']
    title_s = pd.Series(s.index, index = s.values) # 값 <--> 인덱스 변경
    idx = title_s[title]
    
    idx_list = pd.Series(cos_sim[idx].reshape(-1)).sort_values(ascending = False).index[1:topn+1] # 0번재는 본인. 1~10번째
    
    title_list = df.loc[idx_list,'title'].values
    
    return title_list
    

In [77]:
my_search_cossim_by_search4('Toy Story')

array(['Toy Story 3', 'Toy Story 2', 'Toy Story of Terror!', 'Cars 2',
       'Cars', 'Monster House', 'Mr. Warmth: The Don Rickles Project',
       'Luxo Jr.', "A Bug's Life", 'Larry Crowne'], dtype=object)